# Домашнее задание №1: Тональность

Строим модель оценки тональности для отзывов из интернета.

Для этого импортируем нужные библиотеки:

In [3]:
from collections import Counter
import time
import random
import re
from string import punctuation
from bs4 import BeautifulSoup
from nltk import word_tokenize
import requests
import sqlite3
from pymorphy2 import MorphAnalyzer
from fake_useragent import UserAgent

morph = MorphAnalyzer()

Для построения модели выберем отзывы на кофе в зёрнах. Отзывы будем собирать с [сайта Отзывы Тут](https://otzivi-tut.ru).

## Парсинг

Кофе либо сильно хвалят, либо сильно ругают. Попробуем выставить на сайте рейтинг по убыванию/по возрастанию и собрать положительные и отрицательные отзывы. Ссылки на каталог отзывов будут выглядеть вот так:

In [ ]:
# где 2 в конце - номер страницы
desc = 'https://otzivi-tut.ru/category/produkty/kofe_v_zernakh/?sort=rating_desc&PAGEN_1=2'
asc = 'https://otzivi-tut.ru/category/produkty/kofe_v_zernakh/?sort=rating_asc&PAGEN_1=2'

Сами отзывы находятся по ссылке на продукты. Рейтинг продукта определяется количеством звёзд в отзыве (определяется по количеству контейнеров span с классом "star full "). Отзыв содержит также текст достоинств и недостатков продукта. Попробуем собирать полный текст отзыва: и плюсы, и минусы.

Пробуем парсить сайт:

In [ ]:
def parse_site(url, ua):

    """
    Парсинг сайта (html-код) через BeautifulSoup.
    """

    res = requests.get(url, headers={'User-Agent': ua})
    page = res.text
    soup = BeautifulSoup(page, 'html.parser')

    return soup

In [ ]:
ua_1 = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/89.0'
ua_2 = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:91.0) Gecko/20100101 Firefox/91.0'
ua_3 = 'Mozilla/5.0 (Windows NT 6.3; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0'
ua_4 = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36'
ua_5 = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.43 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36 OPR/77.0.4054.277'

list_uas = [ua_1, ua_2, ua_3, ua_4, ua_5]

In [24]:
# пробуем забрать ссылку на продукт

soup_list_prod = parse_site(desc, ua_1)
# даже если в коде страницы есть пробелы после класса, их там нет
list_man_sent = soup_list_prod.find('div', {'class': 'main-sentences-list'}) # контейнер для блоков продуктов
picture_block = list_man_sent.find('div', {'class': 'left-item-block'}) # блок с картинкой
prod_a = picture_block.find('a') # блок ссылки
prod = prod_a.attrs['href']

prod

'/products/245828/kofe_v_zernakh/kofe-v-zernah-miscela-doro-gran-crema/'

In [25]:
# пробуем достать отзыв

prod_url = 'https://otzivi-tut.ru' + prod
soup_prod = parse_site(prod_url, ua_2)
rev_block = soup_prod.find('div', {'class': 'rew-item'})
stars_block = rev_block.find('span', {'class': 'stars'})
list_full_stars = stars_block.find_all('span', {'class': 'star full'})

count_stars = 0
for star in list_full_stars:
    count_stars += 1

rev_text_block = rev_block.find('div', {'class': 'text-s'})
rev_text = rev_text_block.text

print('Рейтинг по отзыву:', count_stars)
print('Текст отзыва:', rev_text)

Рейтинг по отзыву: 3
Текст отзыва: 

Достоинства: Кофе хороший, ароматный, с насыщенным вкусом Недостатки: идет в полудробленом виде, весь пакет такой. На вкус, конечно, не влияет, но кофемашина в этих кусочках захлебывается. Намучался. Комментарий: Вкусно! но не рекомендую, из-за вот этих вот кусочков. Конечно, если есть желание постоянно с кофемашиной стоять и в бункере ножом (любой другой палочкой) шевелить - эт пожалуйста. Но по мне оно такого не стоит.
                                                                                                




Теперь собираем отзывы для обучения и тестирования модели.

In [ ]:
def parse_prod_urls(ctlg_url, ua) -> list:

    """
    Функция, собирающая ссылки на продукты на сайте.
    """

    soup_list_prod = parse_site(ctlg_url, ua)
    list_man_sent = soup_list_prod.find('div', {'class': 'main-sentences-list'})
    picture_blocks = list_man_sent.find_all('div', {'class': 'left-item-block'})
    list_prods_hrefs = [block.find('a').attrs['href'] for block in picture_blocks]

    return list_prods_hrefs

In [ ]:
def parse_reviews(list_prods, list_uas) -> list:
    
    """
    Функция, собирающая отзывы на продукты по ссылкам с сайта.
    """

    list_reviews = []

    for prod in list_prods:

        time.sleep(random.randint(3, 6))
        
        prod_url = 'https://otzivi-tut.ru' + prod
        soup_prod = parse_site(prod_url, random.choice(list_uas))
        rev_blocks = soup_prod.find_all('div', {'class': 'rew-item'})
        # print(prod_url)

        if rev_blocks:

            for rev_block in rev_blocks:

                list_rev = []

                stars_block = rev_block.find('span', {'class': 'stars'})
                list_full_stars = stars_block.find_all('span', {'class': 'star full'})
                count_stars = len(list_full_stars)

                rev_text_block = rev_block.find('div', {'class': 'text-s'})
                rev_text = rev_text_block.text

                list_rev.append(count_stars)
                list_rev.append(rev_text)

                list_reviews.append(list_rev)

    return list_reviews

In [ ]:
list_hrefs = []

for i in range(1, 6):
    time.sleep(random.randint(1, 5))
    desc = f'https://otzivi-tut.ru/category/produkty/kofe_v_zernakh/?sort=rating_desc&PAGEN_1={i}'
    print('Новая страница', desc)
    list_desc_hrefs = parse_prod_urls(desc, random.choice(list_uas))
    list_hrefs.extend(list_desc_hrefs)
    print(list_desc_hrefs[0])

for i in range(1, 6):
    time.sleep(random.randint(3, 7))
    asc = f'https://otzivi-tut.ru/category/produkty/kofe_v_zernakh/?sort=rating_asc&PAGEN_1={i}'
    print('Новая страница', asc)
    list_asc_hrefs = parse_prod_urls(asc, random.choice(list_uas))
    list_hrefs.extend(list_asc_hrefs)
    print(list_asc_hrefs[0])

list_hrefs[:10]

In [ ]:
list_rev_rait = parse_reviews(list_hrefs, list_uas)

In [29]:
list_revs = list_rev_rait

In [30]:
print('Всего отзывов', len(list_revs))

Всего отзывов 414


In [31]:
print('Худшие', list_revs[:5])

Худшие [[1, '\n\nДостоинства: Отвратительный вкус Недостатки: Нет даже намека на вкус кофе Ирландский крем, зерна пережарены, ни при помоле, ни при варке не пахнет кофе. Очень люблю кофе, это худший из всех.\n                                                                                                \n\n'], [1, '\n\nНедостатки: Крупные рыхлые кофейные зерна со слабым запахом. При варке теряет и этот минимальный аромат\n                                                                                                \n\n'], [1, '\n\nДостоинства: Отсутствую Недостатки: Это подделка! Кислятина неимоверная. Хуже самой дешевой робусты. Комментарий: Назвать кофем можно с натяжкой. Полное разочарование.\n                                                                                                \n\n'], [1, '\n\nДостоинства: ничего Недостатки: Зерно кофе старое, аромата и вкуса "Баварский шоколад" нет и в помине. Комментарий: Сплошной негатив! Крайне не рекомендую к покупке!!!\n         

## Препроцессинг отзывов

Токенизируем, приводим к нижнему регистру, лемматизируем.

* Убираем ли стоп-слова? На мой взгляд, здесь убирать стоп-слова не стоит, так как это может повлиять на оценку отзыва (*люблю* - *не люблю*). Тем не менее скорее всего при составлении "тонального словаря" они всё равно уйдут.

* Убираем ли *достоинство* / *недостаток* / *комментарий*? Нет, по причине будущего составления множеств уникальных для отзывов элементов.

In [39]:
list_processed_revs = []

for rev in list_revs:
    rev_rat, rev_text = rev
    rev_text = rev_text.lower()
    rev_text = re.sub('\n+', '', rev_text)
    rev_text = re.sub('\s+', ' ', rev_text)

    lemmas = []
    for word in word_tokenize(rev_text):
        word = word.lower()
        if word not in punctuation and word.isalpha():
            lemma = morph.parse(word)[0].normal_form
            lemmas.append(lemma)

    processed_text = ' '.join(lemmas)
    list_rev = rev_text, rev_rat, processed_text
    list_processed_revs.append(list_rev)

list_processed_revs[0]

('достоинства: отвратительный вкус недостатки: нет даже намека на вкус кофе ирландский крем, зерна пережарены, ни при помоле, ни при варке не пахнет кофе. очень люблю кофе, это худший из всех. ',
 1,
 'достоинство отвратительный вкус недостаток нет даже намёк на вкус кофе ирландский крем зерно пережарить ни при помол ни при варка не пахнуть кофе очень любить кофе это плохой из весь')

## База данных

Будем хранить все данные в базе. Туда будем добавлять текст отзыва и его оценку.

In [5]:
con = sqlite3.connect('coffee_reviews.db')
cur = con.cursor()

In [41]:
cur.execute("""
CREATE TABLE IF NOT EXISTS reviews
(id INTEGER PRIMARY KEY AUTOINCREMENT,
review text,
processed_review text,
rating int,
tonality text)
""")

Загружаем данные в базу. Сразу припишем отзывам тональность, то есть переведём оценку из звёздочек-баллов в словарную оценку. Таким образом, отзывы с оценками 4-5 - это положительные отзывы, 3 - нейтральные отзывы, 1-2 - отрицательные отзывы.

In [42]:
for rev in list_processed_revs:
    rev_text, rev_rat, processed_text = rev
    if rev_rat in [1, 2]:
        ton = 'neg'
    if rev_rat == 3:
        ton = 'neut'
    if rev_rat in [4, 5]:
        ton = 'posit'
    cur.execute(
        '''INSERT INTO reviews (review, processed_review, rating, tonality)
        VALUES (?, ?, ?, ?)''',
        (rev_text, processed_text, rev_rat, ton)
    )

con.commit()

## Тональный словарь

Разберёмся, какие данные у нас будут для обучения, а какие для тестирования.

Так как отзывов в наличии достаточно много, отберём только те, у которых в рейтинге 1 или 5. Для обучения возьмём по 40 отзывов, а для тестирования 5 отрицательных и 5 положительных.

In [6]:
dict_texts_to_learn = {}
dict_texts_to_test = {}

cur.execute(
    '''SELECT processed_review, rating, tonality
    FROM reviews
    WHERE rating = 1 AND tonality = 'neg'
    '''
)

list_neg_texts = [text for text, _, _ in cur.fetchall()]
dict_texts_to_learn['neg'] = list_neg_texts[:41]
dict_texts_to_test['neg'] = list_neg_texts[42:47]

cur.execute(
    '''SELECT processed_review, rating, tonality
    FROM reviews
    WHERE rating = 5 AND tonality = 'posit'
    '''
)

list_posit_texts = [text for text, _, _ in cur.fetchall()]
dict_texts_to_learn['posit'] = list_posit_texts[:41]
dict_texts_to_test['posit'] = list_posit_texts[42:47]

print('Негативный для обучения:', dict_texts_to_learn['neg'][0])
print('Позитивный для обучения:', dict_texts_to_learn['posit'][0])
print('Негативный для тестирования:', dict_texts_to_test['neg'][0])
print('Позитивный для тестирования:', dict_texts_to_test['posit'][0])

Негативный для обучения: достоинство отвратительный вкус недостаток нет даже намёк на вкус кофе ирландский крем зерно пережарить ни при помол ни при варка не пахнуть кофе очень любить кофе это плохой из весь
Позитивный для обучения: достоинство хороший настоящий кофе мягкий и приятный
Негативный для тестирования: достоинство кофе что на картинка афигенный а присылать совершенно не то что заказывать отвратительный вести себя товарищ за работа компенсация геморрой месяц ожидание хоть вернуть недостаток нет комментарий подсесть на этот кофе большой год назад робуста бодрить арбик давать волшебный аромат короче советовать тем кто любить не просто ароматный кофе а именно бодрящий и ароматный
Позитивный для тестирования: достоинство кофе хороший вкус я понравиться за этот деньга это достойный не кислый и зерно не в мочь рекомендовать недостаток быть бы хорошо зип замок комментарий хороший


Теперь создадим тональный словарь. Для этого воспользуемся словарём Counter и множествами.

In [7]:
dict_counters = {}

for key, value in dict_texts_to_learn.items():
    count_lemmas = Counter()
    for text in value:
        list_lemmas = text.split()
        for lemma in list_lemmas:
            count_lemmas[lemma] += 1
    dict_counters[key] = count_lemmas

print(dict_counters)

{'neg': Counter({'кофе': 40, 'не': 39, 'недостаток': 37, 'комментарий': 29, 'достоинство': 25, 'и': 23, 'вкус': 22, 'в': 22, 'нет': 21, 'на': 15, 'это': 15, 'с': 11, 'зерно': 10, 'запах': 9, 'аромат': 9, 'отвратительный': 8, 'как': 8, 'ни': 7, 'пить': 7, 'что': 7, 'этот': 6, 'рекомендовать': 6, 'по': 6, 'весь': 5, 'ничто': 5, 'для': 5, 'деньга': 5, 'срок': 5, 'брать': 5, 'он': 5, 'обжарка': 5, 'пережарить': 4, 'при': 4, 'из': 4, 'цена': 4, 'никакой': 4, 'гадость': 4, 'горький': 4, 'от': 4, 'арабика': 4, 'год': 4, 'покупать': 4, 'без': 4, 'быть': 4, 'даже': 3, 'очень': 3, 'плохой': 3, 'кофейный': 3, 'отсутствовать': 3, 'дешёвый': 3, 'можно': 3, 'полный': 3, 'крайне': 3, 'к': 3, 'покупка': 3, 'такой': 3, 'я': 3, 'ужасный': 3, 'большой': 3, 'купить': 3, 'завысить': 3, 'о': 3, 'сделать': 3, 'фирма': 3, 'производство': 3, 'если': 3, 'хороший': 3, 'италия': 3, 'один': 3, 'то': 3, 'а': 3, 'напиток': 3, 'годность': 3, 'упаковка': 3, 'оказаться': 3, 'изготовитель': 3, 'помол': 2, 'варка': 2, 'л

По субъективному мнению достаточно много слов, которые могли бы лучше оценивать тональность предложения, встречаются в текстах по одному разу, так что удалять нечастотные слова не будем.

In [8]:
dict_sets = {}

for key, value in dict_counters.items():
    set_lemmas = set(value)
    dict_sets[key] = set_lemmas

In [9]:
set_neg = dict_sets['neg']
set_posit = dict_sets['posit']

only_neg = set_neg.difference(set_posit)
only_posit = set_posit.difference(set_neg)

print('Характерны только для негативных отзывов', only_neg)
print('Характерны только для позитивных отзывов', only_posit)

Характерны только для негативных отзывов {'зачем', 'ничто', 'два', 'цвета', 'итальянский', 'уезжать', 'крафтов', 'оставаться', 'оверпрайс', 'напиток', 'нельзя', 'tchibo', 'соответствующий', 'пепельница', 'тем', 'ранее', 'остальной', 'дело', 'непонятный', 'ирландский', 'терпкость', 'приходить', 'заявить', 'январь', 'декабрь', 'покусать', 'признак', 'отсутствие', 'невыраженный', 'компенсация', 'минимальный', 'синтетик', 'килограмм', 'самоделка', 'изготовить', 'предоплата', 'заканчивпться', 'слабый', 'наш', 'жуткий', 'никогда', 'стоить', 'кухонный', 'промыть', 'заказать', 'март', 'пересушить', 'попадаться', 'разочарование', 'проверить', 'редкостный', 'вода', 'ломаный', 'gold', 'плохой', 'неимоверный', 'прикрепить', 'даже', 'выпускать', 'польский', 'название', 'натяжка', 'жардин', 'они', 'пережареный', 'насыщенность', 'отсутствовать', 'у', 'потратить', 'выбрать', 'изготовитель', 'отвратительный', 'старыф', 'описание', 'потенциальный', 'превратиться', 'сплошной', 'нарекание', 'рыхлый', 'чуд

Теперь займёмся данными для тестирования. Для удобства леммы данных текстов тоже объединим в множества.

In [10]:
dict_lemmas_to_test = {}

for key, value in dict_texts_to_test.items():
    list_texts = []
    for text in value:
        lemmas = text.split()
        lemmas = set(lemmas)
        list_texts.append(lemmas)
    dict_lemmas_to_test[key] = list_texts

## Модель

Пишем функцию для определения тональности отзыва. Для этого будем смотреть, сколько слов, положительных или отрицательных, попалось в том или ином отзыве. Каких слов больше, таким и будет наш отзыв.

In [11]:
def sentiment_analysis(set_lemmas, set_neg, set_posit):

    '''
    Функция для определения тональности предложения.
    Принимает на вход набор лемм и сравнивает их с имеющимися леммами в тональном словаре.
    '''

    num_neg = len(set_lemmas & set_neg)
    num_posit = len(set_lemmas & set_posit)

    if num_neg > num_posit:
        return 'neg'
    elif num_posit > num_neg:
        return 'posit'
    else:
        return 'neut'

In [13]:
dict_results = {}

count = 0
for key, value in dict_lemmas_to_test.items():
    for s in value:
        tonality = sentiment_analysis(s, only_neg, only_posit)
        dict_results[count] = [key, tonality]
        count += 1

        print(f'Реальность {key} --- {tonality} Ответ модели')

Реальность neg --- neg Ответ модели
Реальность neg --- neg Ответ модели
Реальность neg --- neg Ответ модели
Реальность neg --- neg Ответ модели
Реальность neg --- posit Ответ модели
Реальность posit --- neg Ответ модели
Реальность posit --- posit Ответ модели
Реальность posit --- neg Ответ модели
Реальность posit --- neut Ответ модели
Реальность posit --- neg Ответ модели


## Оценка оценок

Для оценки работы модели используем метрику *accuracy* - доля правильных ответов среди всех:

In [14]:
count_true = 0
for value_pair in dict_results.values():
    if value_pair[0] == value_pair[1]:
        count_true += 1

accuracy = count_true / len(dict_results)

print('Аккуратность моделирования составляет', accuracy)

Аккуратность моделирования составляет 0.5


## Работа над ошибками

Модель работает не очень хорошо. По моему мнению, причинами этого стало следующее:
* Очень маленький тональный словарь.
* Очень много примесей, ненужных слов в этих словарях (английские названия, случайно оставшиеся стоп-слова).
* Сами отзывы часто содержали не только оценку продукта, но и, например, сервиса доставки.

Что можно улучшить?
* Сделать большой тональный словарь или взять уже готовый (например, LinisCrowd).
* Почистить сделанный словарь от ненужных слов.
* Использовать в дополнение к словарю n-граммную модель. Это позволит, например, определить разницу между *люблю* и *не люблю*.
